In [ ]:
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as mcolors
import pandas as pd
import numpy as np
from tqdm import tqdm
import datetime
import contextlib
import io
import import_ipynb

import enhanced_genetic_algorithm as ega
import particle_swarm_optimization as pso
import non_dominated_sorting_genetic_algorithm_2 as nsga
import evaluate_assignment as evaluate
import utils.fitness_functions as ff

RUNS = 30
DATASET_FILE = "data/corrected_2022_23_arrangement.csv"

# Evaluate benchmark (self-selected) arrangement
SELF_SELECTED_DIVERSITY, SELF_SELECTED_SATISFACTION = None, None
with contextlib.redirect_stdout(io.StringIO()):
    _, SELF_SELECTED_DIVERSITY, SELF_SELECTED_SATISFACTION = evaluate.evaluate()

# Load dataset
dataset_df = pd.read_csv(DATASET_FILE, sep=";")
ega.DATASET = dataset_df
pso.DATASET = dataset_df
nsga.DATASET = dataset_df

nsga_generations = {}
ga_points, pso_points = [], []

with contextlib.redirect_stdout(io.StringIO()):
    for _ in tqdm(range(RUNS), desc="Running Multi-Objective Algorithms"):
        timestamp = datetime.datetime.now().strftime("run_%Y%m%d_%H%M%S")

        ega.RUN_TIME = timestamp
        ega.BASE_DIR = "output/multi_vs_benchmarks/ega"
        _, ga_arr = ega.execute()
        d, s = ff.evaluate_objectives_separately(ga_arr)
        ga_points.append((d, s))

        pso.RUN_TIME = timestamp
        pso.BASE_DIR = "output/multi_vs_benchmarks/pso"
        _, pso_arr = pso.execute()
        d, s = ff.evaluate_objectives_separately(pso_arr)
        pso_points.append((d, s))

        nsga.RUN_TIME = timestamp
        nsga.BASE_DIR = "output/multi_vs_benchmarks/nsga_ii"
        pareto_log, _ = nsga.execute_nsga_ii()
        for gen, front in pareto_log.items():
            nsga_generations.setdefault(gen, []).extend(front)

# Random baseline
random_arr = None
while (random_arr is None):
    random_arr = pso.create_random_teams(dataset_df)

random_div, random_sat = None, None
with contextlib.redirect_stdout(io.StringIO()):
    random_div, random_sat = ff.evaluate_objectives_separately(random_arr)

# Mean positions for GA and PSO
mean_ga_div = np.mean([x[0] for x in ga_points])
mean_ga_sat = np.mean([x[1] for x in ga_points])
mean_pso_div = np.mean([x[0] for x in pso_points])
mean_pso_sat = np.mean([x[1] for x in pso_points])

# Plot
plt.figure(figsize=(10, 6))
gens = sorted(nsga_generations.keys())
cmap = cm.get_cmap("viridis")
norm = mcolors.Normalize(vmin=min(gens), vmax=max(gens))

for gen in gens:
    front = nsga_generations[gen]
    divs = [div for div, sat in front]
    sats = [sat for div, sat in front]
    color = cmap(norm(gen))
    label = f"Gen {gen}" if gen % 5 == 0 else None
    plt.scatter(sats, divs, alpha=0.5, s=20, color=color, label=label)

plt.scatter([mean_ga_sat], [mean_ga_div], marker="*", s=200, color="black", label="Mean Best GA")
plt.scatter([mean_pso_sat], [mean_pso_div], marker="o", s=180, color="black", label="Mean Best PSO")
plt.scatter([SELF_SELECTED_SATISFACTION], [SELF_SELECTED_DIVERSITY], marker="^", s=140, color="black", label="Self-Selected")
plt.scatter([random_sat], [random_div], marker="D", s=140, color="black", label="Random")

plt.title("NSGA-II vs GA/PSO vs Benchmarks on Original Dataset")
plt.xlabel("Satisfaction Score")
plt.ylabel("Diversity Score")
plt.legend(fontsize="small", ncol=2)
plt.grid(True)
plt.tight_layout()
plt.savefig("figures/fig_nsga_ii_ga_pso_vs_self_selected.png")
plt.show()